In [2]:
from ebooklib import epub
from pdfminer.high_level import extract_text

book = 'LN2_G4_Mod1_Extreme_Sports_MEA - al su eobseum.pdf'

# PDF에서 텍스트 추출
pdf_text = extract_text(book)

# EPUB 파일 생성
book = epub.EpubBook()
book.set_title("Converted EPUB")
book.set_language("en")

# 챕터 추가
chapter = epub.EpubHtml(title="Chapter 1", file_name="chap_1.xhtml", content=pdf_text)
book.add_item(chapter)

# EPUB 저장
epub.write_epub("output.epub", book, {})
print("변환 완료: output.epub")


변환 완료: output.epub


In [ ]:
from pdf2image import convert_from_path

poppler_path = r"C:\Users\YJ\Downloads\poppler-24.08.0\Library\bin"

book = 'LN2_G4_Mod1_Extreme_Sports_MEA - al su eobseum.pdf'
# PDF 페이지를 이미지로 변환
images = convert_from_path(book, dpi=300, poppler_path=poppler_path)

# 변환된 이미지 저장
for i, img in enumerate(images):
    img.save(f"page_{i+1}.png", "PNG")
   

ModuleNotFoundError: No module named 'pytesseract'

In [12]:
 
import pytesseract
from PIL import Image

# Tesseract 실행 경로 수동 설정
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"


# OCR 적용
text = ""
for i in range(len(images)):
    text += pytesseract.image_to_string(Image.open(f"page_{i+1}.png"), lang="eng") + "\n"

# 텍스트 저장
with open("output.txt", "w", encoding="utf-8") as f:
    f.write(text)
from ebooklib import epub

book = epub.EpubBook()
book.set_title("Converted EPUB")
book.set_language("ko")

# 챕터 추가
chapter = epub.EpubHtml(title="Chapter 1", file_name="chap_1.xhtml", content=text)
book.add_item(chapter)

# EPUB 저장
epub.write_epub("output.epub", book, {})
print("EPUB 변환 완료: output.epub")


EPUB 변환 완료: output.epub


In [16]:
import pytesseract
from PIL import Image
import pandas as pd
from tabulate import tabulate
import re

# 1️⃣ 이미지에서 텍스트 추출 (OCR 수행)
image_path = "page_4.png"
image = Image.open(image_path)
raw_text = pytesseract.image_to_string(image)

# 2️⃣ OCR 결과 전처리
# 줄바꿈 및 불필요한 공백 제거
text_lines = [line.strip() for line in raw_text.split("\n") if line.strip()]

# 3️⃣ 필요한 부분만 정리 (패턴 기반으로 데이터 추출)
data = []
current_section = None

for line in text_lines:
    # "MODULE"이 포함된 라인은 섹션명으로 인식
    if "MODULE" in line:
        current_section = line.strip()
        continue

    # 페이지 번호를 기반으로 제목과 내용을 분리
    match = re.match(r"(\d+)\s(.+)", line)
    if match:
        page = match.group(1)  # 페이지 번호
        title = match.group(2)  # 제목
        
        # 기본값 설정
        genre = ""
        skill = ""
        
        # OCR 데이터에서 "Genre" 이후의 부분을 찾아서 추가
        if "Explanation" in title or "Recount" in title:
            genre = title.split()[-1]  # 마지막 단어를 장르로 추출
            title = " ".join(title.split()[:-1])  # 장르 제외하고 제목만 남김

        data.append([page, title, genre, skill, current_section])

# 4️⃣ DataFrame 생성 및 출력
df = pd.DataFrame(data, columns=["Page", "Title", "Genre", "Comprehension Skill", "Module"])

# 5️⃣ 마크다운 표 형태로 변환 및 출력
markdown_table = tabulate(df, headers="keys", tablefmt="github")
print(markdown_table)


|    |   Page | Title                       | Genre   | Comprehension Skill   | Module                       |
|----|--------|-----------------------------|---------|-----------------------|------------------------------|
|  0 |      6 | The X Games                 |         |                       |                              |
|  1 |      8 | Extreme BMX                 |         |                       |                              |
|  2 |     10 | Pegsy Oki                   |         |                       |                              |
|  3 |     12 | What Is Bungee Jumping?     |         |                       |                              |
|  4 |     14 | Big-Wave Surfing            |         |                       |                              |
|  5 |     16 | From One Extreme to Another |         |                       | MODULE 2 Genre Comprehension |
|  6 |     20 | Land-Diving on Vanuatu’s    |         |                       | MODULE 2 Genre Comprehension |
|

In [21]:
import cv2
import pytesseract
from PIL import Image
import pandas as pd
from tabulate import tabulate
import re

# 1️⃣ 이미지 로드 및 전처리
image_path = "page_4.png"
image = cv2.imread(image_path)


# Grayscale 변환 (흑백)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Adaptive Thresholding 적용 (텍스트 대비 향상)
gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                             cv2.THRESH_BINARY, 31, 2)

# Morphology 연산 적용 (텍스트 선명하게)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
gray = cv2.morphologyEx(gray, cv2.MORPH_CLOSE, kernel)
gray = cv2.dilate(gray, kernel, iterations=1)

# OpenCV 이미지를 PIL 이미지로 변환 (핵심 수정 부분)
pil_image = Image.fromarray(gray)

# 2️⃣ OCR 수행 (PSM 모드 최적화)
extracted_text = pytesseract.image_to_string(pil_image, lang="eng", config="--psm 6 --oem 3")

# 결과 출력
print(extracted_text)


extreme Action
MODULE 1 Genre Comprehension
Skill
(4 Take [t to the Limit
What is an extreme sport? Read this explanation to learn Explanation Cause and Effect
about some of the crazy things people do for fun.
(6 The X Games
Have you heard of the X Games? Turn to page 6 to read Explanation Cause and Effect
about a contest that pushes the sport to the limits.
(8) Extreme BMX
Read about street, dirt, vert and flatland BMXing on page 8 — | Explanation
it’s not the same as riding your bike to school!
Peggy Oki
Read this biography to learn about trailblazing skateboarder | Recount (Biography)
Peggy Oki and the amazing Z-Boys.
A (42) What Is Bungee Jumping?
Have you taken the big leap yet? Turn to page 12, and decide | Explanation
if bungee is your thing.
A(14) Big-Wave Surfing
Read this explanation and make up your mind — does Explanation
big-wave surfing sound like fun, or like total insanity?
e e
=»
Pushing the Boundaries
im the Past
MODULE ? Genre Comprehension
Skill
From One Extreme to 

In [13]:
import subprocess

try:
    subprocess.run(["ffmpeg", "-version"], check=True)
    print("✅ ffmpeg 실행 가능!")
except FileNotFoundError:
    print("❌ ffmpeg가 설치되지 않았거나 환경 변수에 설정되지 않았습니다.")


✅ ffmpeg 실행 가능!


In [ ]:
from pydub import AudioSegment

# MP3 파일을 PCM WAV 형식으로 변환
mp3_file = "output.mp3"
wav_file = "output.wav"

audio = AudioSegment.from_mp3(mp3_file)
audio.export(wav_file, format="wav", parameters=["-acodec", "pcm_s16le"])

print(f"MP3 파일을 PCM WAV로 변환 완료: {wav_file}")




FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

In [18]:

import speech_recognition as sr

recognizer = sr.Recognizer()

# WAV 파일 열기
with sr.AudioFile(wav_file) as source:
    audio_data = recognizer.record(source)  # 전체 오디오를 읽음
    text = recognizer.recognize_google(audio_data)  # Google STT 사용

print("변환된 텍스트:", text)

# 단어별 타임스탬프 생성 (대략적)
words = text.split()
word_timestamps = []
audio_length = len(audio) / 1000  # 밀리초(ms) -> 초(s)

for i, word in enumerate(words):
    start_time = round(i * (audio_length / len(words)), 2)  # 단어별 대략적 시작 시간
    word_timestamps.append({"word": word, "time": start_time})

# JSON 파일로 저장
import json
with open("timestamps.json", "w", encoding="utf-8") as f:
    json.dump(word_timestamps, f, indent=4, ensure_ascii=False)

print("단어별 타임스탬프 저장 완료: timestamps.json")


ValueError: Audio file could not be read as PCM WAV, AIFF/AIFF-C, or Native FLAC; check if file is corrupted or in another format

In [ ]:
# SMIL 파일 템플릿
smil_template = """<smil>
  <body>
    <seq>
{}
    </seq>
  </body>
</smil>
"""

# SMIL 형식의 항목 생성
smil_items = []
for entry in word_timestamps:
    smil_items.append(f'      <par><text src="text.xml#id{entry["word"]}"/><audio src="output.mp3" clipBegin="{entry["time"]}s"/></par>')

smil_content = smil_template.format("\n".join(smil_items))

# SMIL 파일 저장
with open("output.smil", "w", encoding="utf-8") as f:
    f.write(smil_content)

print("SMIL 파일 생성 완료: output.smil")


PocketSphinx 모델 경로: c:\Users\YJ\.conda\envs\EPUB\lib\site-packages\pocketsphinx\model
폴더 내용: ['CMakeLists.txt', 'en-us']
